<a href="https://colab.research.google.com/github/Exo-dar/exo/blob/master/TRABAJO_PRACTICO/Trabajo_Practico_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Héctor Fernández Bueno  <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---/tree/master/TrabajoPractico<br>
Google Colab: https://colab.research.google.com/drive/1i6CLElITF73OZc0f-1uZAmLxZjmlSX6m <br>
Problema:
>1. Sesiones de doblaje <br>

Descripción del problema: Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible.

....







                                        

#Modelo
**- ¿Como represento el espacio de soluciones?**

El espacio de soluciones lo vamos a representar como un vector de 30 posiciones, donde cada posición representa una toma de la película y su valor será un entero que representará el día en que se ha planificado grabar dicha toma. De este modo, garantizamos que todos los actores implicados sean programados para grabar el mismo día.

**- ¿Cual es la función de aptitud (fitness)?**

La función de aptitud (fitness) que pretendemos minimizar es el coste por doblar la película entera, este coste viene determinado por la cantidad de veces que cada actor se tiene que desplazar al set de grabación, independientemente de la cantidad de tomas que doble.

Para su calculo, se dividirá la función en dos partes. Por un lado, el numero de días distintos que tiene que acudir cada actor, como hemos comentado previamente y, por otro, añadiremos una penalización a dicha función si algún día se graban más de seis tomas, ya que esta es la restricción del problema.

Por lo tanto, el objetivo es minimizar el coste total de tal manera que no se exceda el limite de seis tomas al día.


**- ¿Como implemento las restricciones?**

Como hemos mencionado anteriormente, la restricción principal consiste en limitar a 6 grabaciones de tomas diarias. Esta restricción se implementará, como hemos mencionado anteriomente, en la función de aptitud (fitness). Para ello, se evalua, para cada día, el número total de tomas asignadas y, si algún día supera el umbral de 6 grabaciones, se aplica una penalización por cada toma extra. De este modo, las soluciones que inclumplen la limitación son altamente penalizadas, haciendolas mucho menos atractivas para el proceso evolutivo.

Por otra parte, también hay una restricción "implicita" y es que los actores de doblaje deben coincidir en las tomas en las que aparecen juntos, pero esta limitación la soluciones "implicitamente" al determinar el vector del espacio de soluciones, ya que, tal y como lo hemos planteado, garantizamos que todos los actores implicados sean programados para grabar el mismo día.

#Análisis
**- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones**

Por un lado, el espacio de soluciones es extremadamente grande ya que pueden haber 30 tomas y 30 días posibles para cada una, es decir 30^30. Además, aún teniendo en cuenta la limitación que establece que no pueden doblarse más de 6 tomas en un día, que reduce las posibilidades significativamente, el espacio de posibilidades sigue siendo del orden de 30^30, es decir, inmenso.

Por otra parte la complejidad del algoritmo, viene determinada por sus operaciones internas que, generalmente, para los algoritmos genéticos viene dado por:  
O(G×P×F).
Donde P es el número de individuos en la población, G el numero de generaciones del algoritmo y F la complejidad de la función de fitness.

En nuestro caso, G es el numero de generaciones que fijaremos manualmente como un parametro (NUM_GENERACIONES), pero que su complejidad es O(1) con una constante multiplicativa que será el valor del parametro; P es el número de individuos de la población que, también, fijaremos manualmente como un parametro (POP_SIZE) y que su complejidad también es O(1) con una constante multiplicativa que será el valor del parametro; y, por último, la función de aptitud (fitness) consiste en recorrer todos los actores (NUM_ACTORES) y todas las tomas (NUM_TOMAS) en dos bucles for, por lo tanto, en nuestro caso es de 30*10=300, pero estos son los parametros que pueden variar en función del problema. Luego, si los llamamos n y m a los valores que representan respectivamente, vemos claramente como la complejidad de esta función será de:


*   O(n) (u O(m), si queremos ser estrictos): Siempre que n >> m o m >> n
*   O(n^2): Siempre que n ≈ m

Por lo tanto, en nuestro problema, el orden complejidad del algoritmo vendrá dado por la función de aptitud (fitness) que es la función con un coste más elevado diferencialmente. Es decir, el orden de complejidad será de O(n) u O(n^2) en función del numero de actores y el numero de tomas, tal y como hemos establecido anteriormente.



#Diseño
**- ¿Que técnica utilizo? ¿Por qué?**

La técnica metaheurística elegida para llevar a cabo el ejercicio es el algoritmo genético.  

Esta técnica ha sido la elegida puesto que:


*   Los algoritmos voraces basados en heurísticas simples, suelen tomar decisiones locales, lo que puede llevar facilmente a soluciones que no sean las optimas por quedarse atrapados en óptimos locales. Por lo que este tipo de tecnicas queda descartada.
*   Otras metaheurísticas, como el recocido simulado o la búsqueda tabú, también podrían haber sido empleados, ya que permiten escapar de optimos locales y pueden abordar el problema.

No obstante, consideramos que el algoritmo genético tiene unas características muy favorables para abordar este tipo de problemas, ya que su naturaleza poblacional facilita la exploración amplia del espacio de soluciones y, gracias a estrategias como el elitismo, resulta óptimo para encontrar soluciones factibles y óptimas. Además, tengo un interés particular en profundizar sobre este tipo de algoritmos, ya que me parecen particularmente interesantes. Por lo que la combinación de estos factores es lo que me ha motivado a elegir esta técnica para abordar este problema, a pesar de que otras técnicas también podrían ofrecer muy buenos resultados.  



In [10]:
#Importamos las librerías a emplear
import numpy as np
import random
import pandas as pd

# Definimos los parámetros del problema
NUM_ACTORES = 10          # Número de actores involucrados en el rodaje
NUM_TOMAS = 30            # Número de tomas  a grabar
MAX_TOMAS_POR_DIA = 6     # Número máximo de tomas a grabar diarias
PENALIZACION = 1000       # Penalización para cada toma extra en un día

# Cargamos los datos desde el fichero excel que nos facilitan. Se leen las filas 3 a 32 y columnas B a K, para poder obtener los datos que nos interesan.
df = pd.read_excel('DatosDoblaje.xlsx', usecols="B:K", skiprows=2, nrows=30, header=None)
participacion = df.to_numpy()

# Definimos los parámetros del algoritmo genético
POP_SIZE = 50             # Número de miembros de la población
NUM_GENERACIONES = 1000   # Número de veces que vamos a iterar el algoritmo
PROB_CRUCE = 0.8          # Probabilidad de que se produzca un cruce de genes
PROB_MUTACION = 0.2       # Probabilidad de que se produzca un matación del gen
ELITISMO = 2              # Número de cromosomas que se preservan en cada generación


# Definimos la función de fitness, que será el coste que querremos minimizar. Se calcula el coste teniendo en cuenta dos componentes:
# El coste de desplazamiento de los actores, para cada actor se suma el numero de días distintos que partipa en las tomas.
# La penalización por grabar más de 6 tomas en un día.
def fitness(cromosoma):
    coste_total = 0

    # Calculamos el coste de desplazamiento. Para ello, contamos el numero de días únicos por actor
    for actor in range(NUM_ACTORES):
        dias_participacion = set()
        for toma in range(NUM_TOMAS):
            if participacion[toma, actor] == 1:
                dias_participacion.add(cromosoma[toma])
        coste_total += len(dias_participacion)

    # Aplicamos una penalización por exceder el límite de 6 tomas por día
    dias, cuentas = np.unique(cromosoma, return_counts=True)
    for cuenta in cuentas:
        if cuenta > MAX_TOMAS_POR_DIA:
            coste_total += (cuenta - MAX_TOMAS_POR_DIA) * PENALIZACION

    return coste_total

# Definimos la función que generará la población inicial de manera aleatoria. Cada posición corresponderá a una toma y contendrá el día
# en el que se graba dicha toma.
def generar_cromosoma():
    return [random.randint(0, NUM_TOMAS - 1) for _ in range(NUM_TOMAS)]

# Definimos la función de elección arbitraria de padre y madre. Esta función tiene mayor probabilidad de seleccionar un individuo,
# cuanto menor es el coste
def seleccion(poblacion, costes):
    aptitudes = np.array([1 / (1 + c) for c in costes])
    suma_apt = np.sum(aptitudes)
    probs = aptitudes / suma_apt
    return random.choices(poblacion, weights=probs, k=2)

# Definimos la función para generar los cruces de genes. Para ello, se selecciona un punto de corte aleatorio y se intercambian los segmentos.
def cruce(padre1, padre2):
    punto = random.randint(1, NUM_TOMAS - 1)
    hijo1 = padre1[:punto] + padre2[punto:]
    hijo2 = padre2[:punto] + padre1[punto:]
    return hijo1, hijo2

# Definimos la función para generar las mutaciones de los genes. En esta se modifica aleatoriamente el día asignado a una toma.
def mutacion(cromosoma):
    idx = random.randint(0, NUM_TOMAS - 1)
    cromosoma[idx] = random.randint(0, NUM_TOMAS - 1)
    return cromosoma

# Función que ejecuta un algoritmo clásico de tipo genético
def algoritmo_genetico():
    # Generamos la población inicial
    poblacion = [generar_cromosoma() for _ in range(POP_SIZE)]

    # Inicializamos la mejor solución y el mejor coste
    mejor_cromosoma = None
    mejor_coste = float('inf')

    # Iteramos durante NUM_GENERACIONES
    for generacion in range(NUM_GENERACIONES):
        # Calculamos el coste de cada gen de la población mediante la función fitness
        costes = [fitness(ind) for ind in poblacion]

        # Actualizamos la mejor solución encontrada
        for ind, c in zip(poblacion, costes):
            if c < mejor_coste:
                mejor_coste = c
                mejor_cromosoma = ind.copy()

        # Ordenamos la población población según su coste (de menor a mayor)
        poblacion_costes = sorted(zip(poblacion, costes), key=lambda x: x[1])

        # Elegimos mediante elitismo a los ELITISMO mejores genes y los llevamos directamente a la siguiente generación
        elites = [ind for ind, _ in poblacion_costes[:ELITISMO]]
        nueva_poblacion = elites.copy()

        # Generamos el resto de la nueva población
        while len(nueva_poblacion) < POP_SIZE:
            padre1, padre2 = seleccion(poblacion, costes)
            if random.random() < PROB_CRUCE:
                hijo1, hijo2 = cruce(padre1, padre2)
            else:
                hijo1, hijo2 = padre1.copy(), padre2.copy()
            if random.random() < PROB_MUTACION:
                hijo1 = mutacion(hijo1)
            if random.random() < PROB_MUTACION:
                hijo2 = mutacion(hijo2)
            nueva_poblacion.extend([hijo1, hijo2])
        poblacion = nueva_poblacion[:POP_SIZE]

    # Devolvemos la mejor solución con su coste asociado
    return mejor_cromosoma, mejor_coste

# Definimos una función en la que mostrar todos los resultados que nos parecen relevantes del algoritmo.
# Estos son: La mejor solución del algoritmo, el coste minimo que obtenemos con la mejor solución, el numero total de días
# empleados en la mejor solución y el número de días que debe asistir cada actor de doblaje.
def mostrar_resultados(solucion, coste_total):
    # En primer lugar, vamos a adapta la lista de la mejor solución ya que esta no tiene los numeros adaptados al total de días.
    # Es decir, para mejorar la legibilidad, adaptamos la lista que tendrá numeros entre 0 y NUM_TOMAS-1 para que el más pequeño,
    # sea el día 1, el siguiente sea el día 2...
    # Para ello obtenemos la lista ordenada de los días unicos en la solución
    sorted_unique = sorted(set(mejor_sol))

    # Creamos un diccionario que asocia cada valor con su ranking
    rank_dict = {value: idx+1 for idx, value in enumerate(sorted_unique)}

    # Finalmente, transformamos la lista original asignando el ranking correspondiente
    mejor_sol_normalized = [rank_dict[n] for n in mejor_sol]

    # Calculamos el número total de días necesarios para la grabación (días únicos en la solución)
    dias_totales = len(set(solucion))

    # Para cada actor, contamos el numero de días que participa en las tomas
    dias_por_actor = {}
    for actor in range(NUM_ACTORES):
        dias_actor = set()
        for toma in range(NUM_TOMAS):
            if participacion[toma, actor] == 1:
                dias_actor.add(solucion[toma])
        dias_por_actor[actor + 1] = len(dias_actor)

    # Mostramos por pantalla toda esta información
    print("Mejor solución encontrada (Cada posición representa una toma y el valor representa el día de grabación):")
    print(mejor_sol_normalized)
    print("Coste:", mejor_valor)
    print(f"Número total de días necesarios para la grabación: {dias_totales}")
    print("Días de grabación por actor:")
    for actor, dias in dias_por_actor.items():
        print(f"  Actor {actor}: {dias} día(s)")
    print(f"Coste total: {coste_total}")


mejor_sol, mejor_valor = algoritmo_genetico()
mostrar_resultados(mejor_sol, mejor_valor)


Mejor solución encontrada (Cada posición representa una toma y el valor representa el día de grabación):
[3, 6, 4, 4, 3, 3, 3, 1, 5, 1, 4, 1, 6, 1, 4, 5, 5, 2, 6, 1, 2, 5, 5, 2, 5, 4, 3, 6, 1, 3]
Coste: 32
Número total de días necesarios para la grabación: 6
Días de grabación por actor:
  Actor 1: 5 día(s)
  Actor 2: 4 día(s)
  Actor 3: 6 día(s)
  Actor 4: 4 día(s)
  Actor 5: 4 día(s)
  Actor 6: 2 día(s)
  Actor 7: 1 día(s)
  Actor 8: 3 día(s)
  Actor 9: 2 día(s)
  Actor 10: 1 día(s)
Coste total: 32


Finalmente, solo comentar que, al ser un metodo metaheurístico no siempre tiene por qué encontrar la solución optima. Por ello, la ultima iteración que hemos ejecutado tiene un coste de 32 y un total de 6 días de grabaciones, pero en las diversas ejecuciones que hemos realizado hemos obtenido desde un coste de 37 con 7 días de rodaje, hasta un coste de 31 con 5 días de rodaje. Siendo el coste de 31 y 5 días de rodaje muy probablemente el óptimo global ya que 5 días * 6 Tomas al día = 30 Tomas, que es el numero de tomas totales que tenemos. Establecemos que es, probablemente el optimo global, porque el numero de días ya no se podría optimizar más, aunque el coste quizá aun pueda tener algún pequeño margen de mejora.